<a href="https://colab.research.google.com/github/Azevedo1996/EngenhariadeDados/blob/main/ETL_acoes_bovespa_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

def read_files(path, name_file, year_date, type_file):
    """
    Função para ler arquivos posicionais da B3.
    """
    file_path = f"{path}{name_file}{year_date}.{type_file}"

    # posições das colunas
    colspecs = [(2, 10),   # data pregão
                (10, 12),  # código BDI
                (12, 24),  # sigla da ação
                (27, 39),  # nome da ação
                (56, 69),  # preço de abertura
                (69, 82),  # preço máximo
                (82, 95),  # preço mínimo
                (108, 121),# preço de fechamento
                (152, 170),# quantidade de negócios
                (170, 188) # volume de negócios
               ]

    # nomes das colunas
    names = ['data_pregao', 'codbdi', 'sigla_acao', 'nome_acao', 
             'preco_abertura', 'preco_maximo', 'preco_minimo', 
             'preco_fechamento', 'qtd_negocios', 'volume_negocios']

    # lendo o arquivo
    df = pd.read_fwf(file_path, colspecs=colspecs, names=names, skiprows=1)

    return df

def filter_stocks(df):
    """
    Filtra ações com código BDI igual a 2.
    """
    df = df[df['codbdi'] == 2]
    df = df.drop(['codbdi'], axis=1)
    return df

def parse_date(df):
    """
    Converte a coluna data_pregao para datetime.
    """
    df['data_pregao'] = pd.to_datetime(df['data_pregao'], format='%Y%m%d')
    return df

def parse_values(df):
    """
    Converte as colunas de preços e volume para float.
    """
    df['preco_abertura'] = (df['preco_abertura'] / 100).astype(float)
    df['preco_maximo'] = (df['preco_maximo'] / 100).astype(float)
    df['preco_minimo'] = (df['preco_minimo'] / 100).astype(float)
    df['preco_fechamento'] = (df['preco_fechamento'] / 100).astype(float)
    return df

def concat_files(path, name_file, year_date, type_file, final_file):
    """
    Lê, filtra e processa vários arquivos da B3 e concatena em um único dataframe.
    """
    for i, y in enumerate(year_date):
        df = read_files(path, name_file, y, type_file)
        df = filter_stocks(df)
        df = parse_date(df)
        df = parse_values(df)

        if i == 0:
            df_final = df
        else:
            df_final = pd.concat([df_final, df])

    df_final.to_csv(f"{path}/{final_file}", index=False)

# Executando programa ETL
year_date = ['2018', '2019', '2020']
path = ''  # coloque o caminho para a pasta onde estão os arquivos
name_file = 'COTAHIST_A'
type_file = 'txt'
final_file = 'all_bovespa.csv'
